## Understand And Apply Large Language Models</h3>
### Brief Introduction to Prompt Engineering with OpenAI</h2>
  Initially Written by Pere Martra
  
  Adapted by William Mobley

In this notebook, we'll explore small prompt engineering techniques and recommendations that help us elicit responses from the models that are better suited to our needs.

But first we'll need to select the model we'll use for our LLM. Try different Models and See how the prompts and responses can change. 
Notice: We have different sized models for the Llama2 model. The end denotes the size of the model 7b is 7 billion, 13b is 13billion.
Mistral is a 7b model and mixtral is a 42b model. 

In [1]:
import ollama
from utils import model_chooser
chooser= model_chooser()

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models  respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})
    response = ollama.chat(model=chooser.Model.value, messages=newcontext)

    return (response['message']['content'])

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))


Thank you for asking! The winner of the Formula One World Championship in 2010 was Sebastian Vettel driving for Red Bull Racing. He won 5 races that season and finished on top of the driver's championship standings with a total of 393 points. It was his first championship victory in F1.


For some large models, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Ah, a fellow F1 enthusiast! *adjusts sunglasses* The answer to your first question is none other than Michael Schumacher, driving for the Scuderia Ferrari team. He won the 2000 Formula 1 World Championship with ease, thanks to his exceptional driving skills and the mighty Ferrari F355. *insert cheers and confetti*

Now, on to your next question: Who won the F1 2011 championship? Well, my friend, it was a fierce competition that year, but in the end, Sebastian Vettel emerged victorious driving for Red Bull Racing. His teammate Mark Webber came close, but ultimately had to settle for second place. *nods*

I hope you're enjoying our little chat as much as I am! Do you have any other questions about the fascinating world of F1? 😊


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Ah, another excellent question from you! *adjusts racing gloves*

The winner of the 2006 Formula 1 World Championship was... Sebastian Vettel! *drumroll* Yes, the same driver who won the championship in 2010. Vettel drove for the Red Bull Racing team, which was then known as Scuderia Toro Rosso.

So, to answer your question, Sebastian Vettel won the F1 2006 championship. *tiers hat*


In [7]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Ah, an expert in F1 you are! *adjusts glasses*

Well, my dear, the winner of the 2019 F1 World Championship is... (drumroll please)... Lewis Hamilton! *crowd cheers* Yes, he and his Mercedes AMG Petronas team dominated the season, winning 11 out of 21 races and securing the constructor's championship as well. It was a truly impressive performance.

But I see you already know this, don't you? *wink* You're here to test my F1 knowledge, and I must say, you're quite good at it! *adjusts glasses again* Do you want to ask me another question or shall we continue with our little game?


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [8]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [9]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes AMG Petronas
Points: 413


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Ah, a fellow F1 fan! *adjusts racing gloves*

The winner of the F1 2006 championship was... Lewis Hamilton! *pumps fist*

Driver: Lewis Hamilton.
Team: McLaren Mercedes.


Few Shots for classification.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))


Sentiment: Neutral
